In [1]:
#Import the acrtuals to compare to model guesses
#nc file
import netCDF4
import numpy as np
import time
import matplotlib.pyplot as plt
import xarray as xr
import dask
print('begin')

#Path for Aqua_Planet, 2 months, No VPB
path_to_file = "/fast/gmooers/Real_Geography_Manuscript/Preprocessed_Data/7_Years_Spaced/full_physics_essentials_valid_month02_targets.nc"
ds = xr.open_dataset(path_to_file)
truths = ds.targets[:, :30].values
lons = ds.lon.values
lats = ds.lat.values
print('halfway')
#Path for Aqua_Planet, 2 months, No VPB
path_to_file = "/fast/gmooers/Real_Geography_Manuscript/Models/Linear_Baseline.nc"
ds = xr.open_dataset(path_to_file)
features = ds.Prediction[:, :30].values

x = 144
#x = 128
y = 96
#y=64
z = 30
t = int(len(truths)/(x*y))
print(t)

begin
halfway
2879


In [3]:
reconstructed_targets = np.zeros(shape=(x, y, t, z))
reconstructed_features = np.zeros(shape=(x, y, t, z))
#print(reconstructed_targets.shape)
print('hi')
count = 0
for i in range(t):
    for j in range(y):
        for k in range(x):
            A = truths[count]
            B = features[count]
            reconstructed_targets[k, j, i, :] = A
            reconstructed_features[k, j, i, :] = B
            count = count + 1

print(reconstructed_targets.shape)
print(reconstructed_features.shape)

hi
(144, 96, 2879, 30)
(144, 96, 2879, 30)


In [4]:
#code for all longitudes 
A = reconstructed_targets
B = reconstructed_features

targs = reconstructed_targets
feats = reconstructed_features


R = np.zeros(shape=(x, y, 1, z))
SSE = np.zeros(shape=(x, y, 1, z))
SVAR = np.zeros(shape=(x, y, 1, z))
target_list = []
feature_list = []
count = 0

for i in range(len(A)):
    for j in range(len(A[0])):
        for k in range(len(A[0][0][0])):
            for l in range(len(A[0][0])):
                target_value = reconstructed_targets[i][j][l][k]
                feature_value = reconstructed_features[i][j][l][k]
                target_list.append(target_value)
                feature_list.append(feature_value)
            
            target_list = np.array(target_list)
            feature_list = np.array(feature_list)
            sse = np.sum((target_list-feature_list)**2.0)
            svar = np.sum((target_list-np.mean(target_list))**2.0)
            r_2 = 1-(sse/svar)
            R[i, j, :, k] = r_2
            SSE[i, j, :, k] = sse
            SVAR[i, j, :, k] = svar
            target_list = []
            feature_list = []
    
    if i%10==0:
        print(i)
            

print('hi')
print(R.shape)
R_Save = np.squeeze(R)
SSE_Save = np.squeeze(SSE)
SVAR_Save = np.squeeze(SVAR)
#np.save("/fast/gmooers/RG_Paper_Data/April_SSE_SPCAM5_15_Min_Interval_Heat.npy", SSE_Save)
#np.save("/fast/gmooers/RG_Paper_Data/April_SVAR_SPCAM5_15_Min_Interval_Heat.npy", SVAR_Save)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Linear_Baseline_15_Min_Interval_Heating.npy", R_Save)
print("Save completed - nothing below here matters")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
hi
(144, 96, 1, 30)
